# Working with Enriched Events data

In this Lab we'll learn how to work with [Enriched Event data](https://docs.developers.optimizely.com/web/docs/enriched-events-export) using [PySpark](http://spark.apache.org/docs/latest/api/python/index.html) and [Spark SQL](http://spark.apache.org/sql/).

This lab contains a set of simple, useful queries for working with this dataset.  These queries can help you answer questions like
- How many visitors were tested in my experiment?
- How many "unique conversions" of an event were attributed to this variation?

This Lab covers some of the basics of working with event-level experiment data. There are many more useful questions may want to answer with experiment data.  Future tutorials will go deeper on the topic.

This guide borrows some initialization code from the [Spark SQL getting started guide](https://spark.apache.org/docs/latest/sql-getting-started.html).

## Import your packages

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os
import pyspark
from itertools import product
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.sql.repl.eagerEval.truncate", 100) \
    .getOrCreate()

## Set your main variables
**Token**: API Token taken from Optimizely, follow instructions [here](https://help.optimizely.com/Integrate_Other_Platforms/Generate_a_personal_access_token_in_Optimizely_X_Web)<br>
**Experiment**: List of experiment ID's you wish to analyze, <ins>if empty will analyze all experiments within a project</ins><br>
**Start_time**: Start date of results analysis. "YYYY-MM-DD" <br>
**End_time**: End date of results analysis. "YYYY-MM-DD" <br>
**Event_list**: A list of events you want to analyze, <ins>if empty will analyze all events received<ins><br>
**Output_file_path**: Path you want to export the data to. 

In [4]:
token = ''

experiment_list = ['experimentId here']

start_time = 'YYYY-MM-DD'
end_time = 'YYYY-MM-DD'

event_list = ['event name here']

output_file_path = '/nameOfFile';

output_dir = os.getcwd() + output_file_path

## Bash Script to Download Decisions

In [5]:
%%bash -s "$token" "{" ".join(experiment_list)}" "$start_time" "$end_time" "$output_dir"

if [[ -z $2 ]]; then
oevents load \
    --token $1 \
    --type decisions \
    --start $3 \
    --end $4 \
    --output $5
else
for e in $2
do
oevents load \
    --token $1 \
    --type decisions \
    --start $3 \
    --end $4 \
    --experiment $e \
    --output $5
done
fi


download: s3://optimizely-events-data/v1/account_id=8253175392/type=decisions/date=2021-09-02/experiment=9300000009180/part-00000-c72736c5-36f7-4f0c-8374-2bfa2addab51.c000.snappy.parquet to autoShortForm/type=decisions/date=2021-09-02/experiment=9300000009180/part-00000-c72736c5-36f7-4f0c-8374-2bfa2addab51.c000.snappy.parquet


## Bash Script to Download Events

In [4]:
%%bash -s "$token" "{" ".join(event_list)}" "$start_time" "$end_time" "$output_dir"
if [[ -z $2 ]]; then
oevents load \
    --token $1 \
    --type events \
    --start $3 \
    --end $4 \
    --output $5
else
for e in $2
do
oevents load \
    --token $1 \
    --type events \
    --start $3 \
    --end $4 \
    --event $e \
    --output $5
done
fi

## Loading Enriched Event Data into Notebook

The `OPTIMIZELY_DATA_DIR` environment variable may be used to specify the local directory where Enriched Event data is stored.  If, for example, you've [downloaded Enriched Event data](https://docs.developers.optimizely.com/optimizely-data/docs/enriched-events-getting-started) and saved it in `optimizely_data` in your home directory, you can load that data in this notebook by executing the following command before launching Jupyter Lab:

```
$ export OPTIMIZELY_DATA_DIR=~/optimizely_data
```

If `OPTIMIZELY_DATA_DIR` is not set, data will be loaded from `./data` in your working directory.

In [6]:
base_data_dir = os.environ.get("OPTIMIZELY_DATA_DIR", "nameOfFile")

def read_data(path, view_name):
    """Read parquet data from the supplied path and create a corresponding temporary view with Spark."""
    spark.read.parquet(path).createOrReplaceTempView(view_name)

In [7]:
decisions_dir = os.path.join(base_data_dir, "type=decisions")
read_data(decisions_dir, "decisions")

In [8]:
spark.sql("""
    SELECT
        *
    FROM decisions
    LIMIT 1
""")


uuid,timestamp,process_timestamp,visitor_id,session_id,account_id,campaign_id,experiment_id,variation_id,attributes,user_ip,user_agent,referer,is_holdback,revision,client_engine,client_version,date,experiment
5be5dadf-4daa-45dd-8fb0-45009be70e48,2021-09-01 13:54:49.915,2021-09-01 13:55:20.336,02548cad-4fd3-4fe7-b65c-3c17f3f369ec,-1391194159,8253175392,9300000009180,9300000009180,6555,"[{20360686443, flow, custom, FLOW.SHORT}, {$opt_bot_filtering, $opt_bot_filtering, custom, false}...",73.8.206.0,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515...",https://www.costcochoice.com/,false,336,javascript-sdk,4.5.1,2021-09-01,9300000009180


In [16]:
spark.sql("""
    SELECT
        *
    FROM decisions
""")

uuid,timestamp,process_timestamp,visitor_id,session_id,account_id,campaign_id,experiment_id,variation_id,attributes,user_ip,user_agent,referer,is_holdback,revision,client_engine,client_version,date,experiment
5be5dadf-4daa-45dd-8fb0-45009be70e48,2021-09-01 13:54:49.915,2021-09-01 13:55:20.336,02548cad-4fd3-4fe7-b65c-3c17f3f369ec,-1391194159,8253175392,9300000009180,9300000009180,6555,"[{20360686443, flow, custom, FLOW.SHORT}, {$opt_bot_filtering, $opt_bot_filtering, custom, false}...",73.8.206.0,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515...",https://www.costcochoice.com/,false,336,javascript-sdk,4.5.1,2021-09-01,9300000009180
f227c334-e984-41aa-a9a4-4747c4b7bd13,2021-08-31 20:38:41.855,2021-08-31 20:39:05.576,056976f2-84d1-43e6-b849-0b3d5ead57d3,-1337509197,8253175392,9300000009180,9300000009180,6555,"[{20360686443, flow, custom, FLOW.SHORT}, {$opt_bot_filtering, $opt_bot_filtering, custom, false}...",24.234.62.0,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515...",https://www.costcochoice.com/,false,322,javascript-sdk,4.5.1,2021-09-01,9300000009180
7ea32f35-e724-427f-b31f-58296e8ae1d6,2021-08-31 20:39:09.537,2021-08-31 20:40:05.475,056976f2-84d1-43e6-b849-0b3d5ead57d3,-1337509197,8253175392,9300000009180,9300000009180,6555,"[{20360686443, flow, custom, FLOW.SHORT}, {$opt_bot_filtering, $opt_bot_filtering, custom, false}...",24.234.62.0,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515...",https://www.costcochoice.com/,false,322,javascript-sdk,4.5.1,2021-09-01,9300000009180
ecdaed53-c59f-440b-8223-563503c625ad,2021-08-31 20:39:09.607,2021-08-31 20:40:05.475,056976f2-84d1-43e6-b849-0b3d5ead57d3,-1337509197,8253175392,9300000009180,9300000009180,6555,"[{20360686443, flow, custom, FLOW.SHORT}, {$opt_bot_filtering, $opt_bot_filtering, custom, false}...",24.234.62.0,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515...",https://www.costcochoice.com/,false,322,javascript-sdk,4.5.1,2021-09-01,9300000009180
8da14434-1abb-4ada-98d8-34bf27232b1c,2021-08-31 20:39:09.634,2021-08-31 20:40:05.475,056976f2-84d1-43e6-b849-0b3d5ead57d3,-1337509197,8253175392,9300000009180,9300000009180,6555,"[{20360686443, flow, custom, FLOW.SHORT}, {$opt_bot_filtering, $opt_bot_filtering, custom, false}...",24.234.62.0,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515...",https://www.costcochoice.com/,false,322,javascript-sdk,4.5.1,2021-09-01,9300000009180
c2c0cbf6-16ae-42e2-ad1c-e95f1079e447,2021-08-31 20:39:10.189,2021-08-31 20:40:05.475,056976f2-84d1-43e6-b849-0b3d5ead57d3,-1337509197,8253175392,9300000009180,9300000009180,6555,"[{20360686443, flow, custom, FLOW.SHORT}, {$opt_bot_filtering, $opt_bot_filtering, custom, false}...",24.234.62.0,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515...",https://www.costcochoice.com/,false,322,javascript-sdk,4.5.1,2021-09-01,9300000009180
fe0eb371-8e94-4f80-9dac-93d7ac4584b8,2021-08-31 20:39:10.202,2021-08-31 20:40:05.475,056976f2-84d1-43e6-b849-0b3d5ead57d3,-1337509197,8253175392,9300000009180,9300000009180,6555,"[{20360686443, flow, custom, FLOW.SHORT}, {$opt_bot_filtering, $opt_bot_filtering, custom, false}...",24.234.62.0,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515...",https://www.costcochoice.com/,false,322,javascript-sdk,4.5.1,2021-09-01,9300000009180
569d8914-f8bb-461f-95ae-f26a3da02d8e,2021-08-31 20:39:11.598,2021-08-31 20:40:05.475,056976f2-84d1-43e6-b849-0b3d5ead57d3,-1337509197,8253175392,9300000009180,9300000009180,6555,"[{20360686443, flow, custom, FLOW.SHORT}, {$opt_bot_filtering, $opt_bot_filtering, custom, false}...",24.234.62.0,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515...",https://www.costcochoi

## Converting to CSV file

In [2]:
conf = pyspark.SparkConf()
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '45G')
        .set('spark.driver.maxResultSize', '10G'))

sc = pyspark.SparkContext.getOrCreate(conf=conf)
sqlcontext = pyspark.SQLContext(sc)

In [6]:
def return_e3_dataframe(data_type="events"):
    """Input some info here"""
    data_type = "/type=events" if data_type == "events" else "/type=decisions"
    dataframes = []
    for x in os.walk(output_dir + data_type):
        for i in x:
            if type(i) is list:
                for j in i:
                    if str.endswith(j,'parquet'):
                        print(x[0] + '/' + j)
                        pf = sqlcontext.read.parquet(x[0] + '/' + j)
                        df = pf.toPandas()
                        dataframes.append(df)
                        df = None
                        pf = None
    pdf = pd.concat(dataframes)
    test_data = pdf if data_type=='/type=decisions' else pdf[pdf.experiments.apply(lambda tuple_list: any(check in experiment_list for check in [x.experiment_id for x in tuple_list if x.is_holdback==False]))]
    return test_data

Enriched Event data is partitioned into two distinct datasets: [decisions](https://docs.developers.optimizely.com/optimizely-data/docs/enriched-events-data-specification#decisions-2) and [conversions](https://docs.developers.optimizely.com/optimizely-data/docs/enriched-events-data-specification#conversions-2).

We'll load decision data from the `type=decisions` directory in the base data directory.

In [21]:
#events_df = return_e3_dataframe()
decisions_df = return_e3_dataframe(data_type='decisions')

#events_df.to_csv('autoShortForm_events.csv')
decisions_df.to_csv('autoShortForm_decisions.csv')


/Users/rsaunders/code/jupyter/my_work/autoShortForm/type=decisions/date=2021-09-01/experiment=9300000009180/part-00000-bb25b682-cf7a-4123-98e3-242968c501bf.c000.snappy.parquet
/Users/rsaunders/code/jupyter/my_work/autoShortForm/type=decisions/date=2021-08-30/experiment=9300000009180/part-00000-eb4b675f-d611-4d71-b72c-a5e2c64f8a32.c000.snappy.parquet
/Users/rsaunders/code/jupyter/my_work/autoShortForm/type=decisions/date=2021-08-31/experiment=9300000009180/part-00000-da977cfc-0f0e-434d-9095-804d601c6b11.c000.snappy.parquet
/Users/rsaunders/code/jupyter/my_work/autoShortForm/type=decisions/date=2021-08-28/experiment=9300000009180/part-00000-fb06b3f9-7a03-42cd-a2a6-f2e44fa22492.c000.snappy.parquet
/Users/rsaunders/code/jupyter/my_work/autoShortForm/type=decisions/date=2021-08-26/experiment=9300000009180/part-00000-8e85bc34-5b13-498f-9efa-0627f70743eb.c000.snappy.parquet
/Users/rsaunders/code/jupyter/my_work/autoShortForm/type=decisions/date=2021-08-27/experiment=9300000009180/part-00000-0

In [9]:
spark.sql("""
    SELECT
        experiment_id,
        variation_id,
        COUNT(distinct visitor_id) as `Auto Short Form, iPhone Unique visitors`
    FROM decisions
    WHERE 
        user_agent like '%iPhone%'
    GROUP BY
        experiment_id,
        variation_id
""")

experiment_id,variation_id,"Auto Short Form, iPhone Unique visitors"
9300000009180,6555,2


In [10]:
spark.sql("""
    SELECT
        experiment_id,
        variation_id,
        COUNT(distinct visitor_id) as `Auto Short Form, Android Unique visitors`
    FROM decisions
    WHERE 
        user_agent like '%Android%'
    GROUP BY
        experiment_id,
        variation_id
""")

experiment_id,variation_id,"Auto Short Form, Android Unique visitors"
9300000009180,6555,27
